# Lesson 6 - TensorFlow and Keras

## 6.6.1 History of Artificial Neural Networks

Most of the history is inapplicable to practical skills that these notes focus on. Basically, ANNs were not used until recently when GoogleBrain made it popular again. A key tool used by GoogleBrain was DistBelief, and its popularity rose due to its open source nature.

In 2015, DistBelief was changed in a new iteration, becoming __TensorFlow.__ The competitor-tool to TensorFlow was the non-Google __Keras__. Nowadays, howver, Keras and TensorFlow are integrated, allowing Keras to be used natively with TensorFlow.

## 6.6.2 How TensorFlow Works

Tensors are array that have ___ranks.___ Ranks represent the number of dimensions of a tensor-array (or simply, a "tensor"). To convert a tensor to an actual NumPy array, use `.eval()`.

In [1]:
# the main import
import tensorflow as tf


In [2]:
[3, 2, 1]                  # rank 1 (single dimensional vector)
[[3, 2], [1, 3]]           # rank 2 (two dimensions)
[[[1], [2]], [[1], [2]]]   # rank 3
2                          # rank 0 (scalar values have no dimensionality)

2

### Nodes

__Thinkful Definition:__ "Key object that is a place where things can happen in our model." Cool.

#### Node - The "Constant"

In [2]:
node_const = tf.constant(70)
print(node_const) # notice how it prints the node itself, not the constant "70"

Tensor("Const:0", shape=(), dtype=int32)


#### Node - The "Mathematical Operator" (Add, Multiply, etc.)

In [7]:
node_add = tf.add(node_const, node_const)

print(node_add) # every time you re-run it, the name changes to Add_n+1
# The value (0) doesn't change because the rank of the constant is always 0

Tensor("Add_3:0", shape=(), dtype=int32)


#### Node - The "Placeholder"

In [8]:
node_place = tf.placeholder(tf.int32)

print(node_place)

Tensor("Placeholder_1:0", dtype=int32)


#### Node - The "Variable"

Has the properties of a placeholder (it literally "holds the place of a value") but with additional features, making it have a _variable value_ instead of a placeholder's constant value.

Also, you need to manually initialize them (ugh).

In [12]:
q = tf.Variable([0], tf.float32)
init = tf.global_variables_initializer()
sess.run(init)

Instructions for updating:
Colocations handled automatically by placer.


### Sessions

Instead of simply identifying the node, "sessions" actually "runs" the node and generates an output.

In [9]:
sess = tf.Session()

sess.run(node_const)

70

In [10]:
# utilizing all nodes as an example

a = tf.placeholder(tf.int32)

# Create an operator node that takes our placeholder and a constant node
multiply_by_2 = tf.multiply(a, tf.constant(2))

# Run the node to return our output
sess.run(multiply_by_2, {a : 3}) # this is what you use a placeholder for

6

In [11]:
# the beauty of tensors is that it performs higher matrix operations for you!
# (something good for image data analysis? (wink hint wink?))
sess.run(multiply_by_2, {a : [[3, 4, 81], [2, 31, 13]]})

array([[  6,   8, 162],
       [  4,  62,  26]])

## 6.6.3 - Example TensorFlow Model

In [13]:
# setting variables
# Note that our initial value has to match the data type
# so 1 would give an error since it's an int...
b = tf.Variable([1.], tf.float32)
m = tf.Variable([1.], tf.float32)
x = tf.placeholder(tf.float32)

# Implement a linear model with shorthand for tf.add() by using '+'
# and tf.multiply with '*'
linear_model = m * x + b

# New variables means we have to initialize again
init = tf.global_variables_initializer()
sess.run(init)

In [14]:
print(sess.run(linear_model, {x:[1, 2, 3, 4]}))

[2. 3. 4. 5.]


In [15]:
# getting fancier...
# creating a loss function and thereby determining accuracy

y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1, 2, 3, 4], y:[.1, -.9, -1.9, -2.9]}))

# That's a high sum squared loss! Large error!

116.04001


In [16]:
# You can go back and adjust the model with the node
#########TF.ASSIGN!

fixm = tf.assign(m, [-1.])
fixb = tf.assign(b, [1.1])
sess.run([fixm, fixb])
print(sess.run(loss, {x:[1, 2, 3, 4], y:[.1, -.9, -1.9, -2.9]}))

4.9960036e-16


^^^The error is basically zero here, which is great!<br>
But how did we know to re-assign m and b to be -1 and 1.1, respectively? Thinkful cheated...

What we _can_ do is to create a gradient descent function to minimize the loss function. We can set the gradient descent to be a tensorflow node.

In [17]:
# Set your learning rate in Gradient Descent - 0.01 is just fine
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# reset values to incorrect defaults.
# Otherwise, the "session" would work off of itself - VERY IMPORTANT!*******
sess.run(init) 

# Loop for 100 iterations, trying to find optimal values
for i in range(100):
    sess.run(train, {x:[1, 2, 3, 4], y:[.1, -.9, -1.9, -2.9]})

print(sess.run([m, b]))

Instructions for updating:
Use tf.cast instead.
[array([-0.9286998], dtype=float32), array([0.89036876], dtype=float32)]


## 6.6.4 Keras Introduction

Keras is more accessible but can do less than TensorFlow (according to Thinkful, of course). Instead of TensorFlow's "nodes" and "variables", Keras has:

__Layers:__ Like layers in a typical neural network model (a set of perceptron nodes).
- A layer is called a "dense" layer if every node connects to every node in the next layer.

__Models:__ The structure of your layering. You "make a model" by stacking the layers (see example below). Two flavors of model:
- _Sequential_: Stacks of layers in a linear progression.
- _Complex_: Non-sequential layering. Obviously such a model is more...ummm...'complex'.

In [19]:
from keras.models import Sequential 

model = Sequential()
from keras.layers import Dense, Activation

model.add(Dense(units=100, input_dim=100)) # "adding" on top of one another 
model.add(Activation('relu'))              # forms a sequential model
model.add(Dense(units=10))
model.add(Activation('softmax'))

Using TensorFlow backend.


In [ ]:
# ...I did it.

